[![Notebook Tutorial](__code/__all/notebook_tutorial.png)](https://neutronimaging.pages.ornl.gov/en/tutorial/notebooks/water_intake_profile_calculator/)

# Select your IPTS 

<img src='__docs/__all/shift_enter.gif' />

In [1]:
from __code import system
from __code.water_intake_profile_calculator import WaterIntakeProfileCalculator, WaterIntakeProfileSelector

from __code.ui_builder import UiBuilder
o_builder = UiBuilder(ui_name = 'ui_water_intake_profile.ui')
from __code.roi_selection_ui import Interface

system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

pyuic5 /Users/j35/git/IPTS/python_notebooks/ui/ui_water_intake_profile.ui -o /Users/j35/git/IPTS/python_notebooks/__code/ui_water_intake_profile.py


** Using Debugging Mode! **


VBox(children=(HBox(children=(Label(value='IPTS-'), Text(value='', layout=Layout(width='10%')), Label(value='DOES NOT EXIST!', layout=Layout(width='20%'), _dom_classes=('result_label',)))), Label(value='OR'), HBox(children=(Label(value='Select Folder', layout=Layout(width='20%')), Select(layout=Layout(height='300px'), options=('IPTS-15177', 'IPTS-18813', 'Richard'), value='IPTS-15177'))), Button(button_style='info', description='HELP', style=ButtonStyle())))

# Description 

This notebook will calculate the water intake profile vs time of a sample.

Here are the steps (**bold** for user input/manipulation)
 * **select the normalized images**
 * images sorted by time
 * **region of interest seleted**
 * profile of counts vs vertical-pixel calculated (**select integrated algorithm: mean/median/sum**)
 * water intake profile vs file index or vs time
 * **export profiles**

# Python Import 

In [2]:
%gui qt

# Select Images to Process

In [3]:
o_water = WaterIntakeProfileCalculator(working_dir = system.System.get_working_dir())
o_water.select_data()

Button(button_style='info', description='HELP', style=ButtonStyle())

HTML(value='Please wait...')

VBox(children=(Label(value='Select Images ...', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Users/j35/IPTS/Richard', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' .DS_Store     |     Wed Feb 28 09:11:21 2018', ' Day1          |     Directory'), value=()), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

HTML(value='Please wait...')

VBox(children=(Label(value='Select Images ...', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Users/j35/IPTS/Richard/Day1', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' 20180129_Attenuation_ss_ls_sh_0040_2551.tiff          |     Mon Jan 29 10:51:48 2018', ' 20180129_BanderaGrey_Dolomite_Dry_0040_0182.tiff      |     Mon Jan 29 16:04:00 2018', ' 20180129_BanderaGrey_Dolomite_Dry_0040_0183.tiff      |     Mon Jan 29 16:04:45 2018', ' 20180129_BanderaGrey_Dolomite_Dry_0040_0184.tiff      |     Mon Jan 29 16:05:30 2018', ' 20180129_BanderaGrey_Dolomite_Dry_0040_0185.tiff      |     Mon Jan 29 16:06:14 2018', ' 20180129_BanderaGrey_Dolomite_Dry_0040_0186.tiff      |     Mon Jan 29 16:06:59 2018', ' 20180129_BanderaGrey_Dolomite_Dry_0040_0187.tiff      |     Mon Jan 29 16:07:44 2018', ' 20180129_BanderaGrey_Dolomite_Dry_0040_0188.tiff      |     Mon Jan 29 16:08:29 2018', ' 20180129_BanderaGrey_Dolomite_Dry_0040_0189.tiff      |     Mon Jan 29 16:09:14 2018', ' 20180129_BanderaGrey_Dolomite_Dry_0040_0190.tiff      |     Mon Jan 29 16:09:59 2018', ' 20180129_BanderaGrey_Dolomite_Dry_0040_0191.tiff      |     Mon Jan 29 16:10:44 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0215.tiff     |     Mon Jan 29 17:12:26 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0216.tiff     |     Mon Jan 29 17:13:11 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0217.tiff     |     Mon Jan 29 17:13:55 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0218.tiff     |     Mon Jan 29 17:14:40 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0219.tiff     |     Mon Jan 29 17:15:25 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0220.tiff     |     Mon Jan 29 17:16:10 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0221.tiff     |     Mon Jan 29 17:16:55 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0222.tiff     |     Mon Jan 29 17:17:40 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0223.tiff     |     Mon Jan 29 17:18:25 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0224.tiff     |     Mon Jan 29 17:19:10 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0225.tiff     |     Mon Jan 29 17:19:55 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0226.tiff     |     Mon Jan 29 17:20:40 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0227.tiff     |     Mon Jan 29 17:21:25 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0228.tiff     |     Mon Jan 29 17:22:09 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0229.tiff     |     Mon Jan 29 17:22:54 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0230.tiff     |     Mon Jan 29 17:23:39 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0231.tiff     |     Mon Jan 29 17:24:24 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0232.tiff     |     Mon Jan 29 17:25:09 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0233.tiff     |     Mon Jan 29 17:25:54 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0234.tiff     |     Mon Jan 29 17:26:39 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0235.tiff     |     Mon Jan 29 17:27:24 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0236.tiff     |     Mon Jan 29 17:28:09 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0237.tiff     |     Mon Jan 29 17:28:54 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0238.tiff     |     Mon Jan 29 17:29:38 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0239.tiff     |     Mon Jan 29 17:30:23 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0240.tiff     |     Mon Jan 29 17:31:08 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0241.tiff     |     Mon Jan 29 17:31:53 2018', ' 20180129_BanderaGrey_Dolomite_Wet3_0040_0242.tiff     |     M

HBox(children=(Label(value='Retrieving Time Stamp', layout=Layout(width='20%')), IntProgress(value=0, layout=Layout(width='50%'), max=4)))

HBox(children=(Label(value='Loading sample', layout=Layout(width='20%')), IntProgress(value=0, max=4)))

# Select Profile Region 

In [6]:
o_gui = WaterIntakeProfileSelector(dict_data=o_water.dict_files)
o_gui.show()